In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp /content/gdrive/My\ Drive/RecSys -r ./
%cd RecSys

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/RecSys


### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [2]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

In [ ]:
!pip install implicit
!pip install lightfm

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [ ]:
!unzip ml-1m.zip

In [4]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [5]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [ ]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [6]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [ ]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [159]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True, use_gpu=False)

В качестве loss здесь всеми любимый RMSE

In [ ]:
model.fit(user_item_t_csr)

In [ ]:
len(model.item_factors)

3953

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [ ]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [111]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [ ]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '33    Babe (1995)',
 '360    Lion King, The (1994)',
 '2315    Babe: Pig in the City (1998)',
 '1526    Hercules (1997)',
 '1838    Mulan (1998)',
 '2618    Tarzan (1999)']

Tensorboard visualization

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

%load_ext tensorboard

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector

VISUAL_LOGDIR = 'ml-1m/visual/'

def create_visualization(model, name: str, log_dir=VISUAL_LOGDIR):
    log_dir = os.path.join(log_dir, name)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    mv2data = {}
    for id, name in zip(movie_info['movie_id'], movie_info['name']):
        mv2data[id] = name
    for id, vec in enumerate(model.item_factors):
        mv2data[id] = (mv2data.get(id, 'unknown_' + str(id)), vec)

    with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
        for key in mv2data:
            f.write("{}\n".format(mv2data[key][0]))

    weights = tf.Variable(model.item_factors)
    checkpoint = tf.train.Checkpoint(embedding=weights)
    checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = 'metadata.tsv'
    projector.visualize_embeddings(log_dir, config)

In [ ]:
create_visualization(model, 'als_implicit')

In [ ]:
%tensorboard --logdir 'ml-1m/visual/als_implicit/'

Давайте теперь построим рекомендации для юзеров

Как мы видим, юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [112]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [ ]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [113]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [ ]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '2502    Matrix, The (1999)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1182    Aliens (1986)',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '2460    Planet of the Apes (1968)',
 '1884    French Connection, The (1971)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Всякие нужные вещи

In [110]:
# Constants

MOVIE_ID = 1
USER_ID = 4

N_EPOCHS = 10
N_FACTORS = 64
LEARNING_RATE = 0.01

In [121]:
from typing import Dict, List, Tuple

class RecModule:
    def __init__(self, n_factors=100, lr=0.1, user_mu=0.01, item_mu=0.01):
        self.n_factors = n_factors
        self.lr = lr
        self.user_mu = user_mu
        self.item_mu = item_mu

        self.n_users = 0
        self.n_items = 0

        self.user_factors = None
        self.item_factors = None
        self.user_bias = None
        self.item_bias = None
        self.global_bias = 0.

    def _init_vectors(self):
        self.user_factors = np.random.normal(0., 1./np.sqrt(self.n_factors),
                                             size=(self.n_users, self.n_factors))
        self.item_factors = np.random.normal(0., 1./np.sqrt(self.n_factors),
                                             size=(self.n_items, self.n_factors))
        self.user_bias = np.zeros((self.n_users,), dtype=np.float32)
        self.item_bias = np.zeros((self.n_items,), dtype=np.float32)

    def predict_score(self, user_id: int, item_id: int) -> float:
        res = np.dot(self.user_factors[user_id], self.item_factors[item_id])
        res += self.global_bias + self.user_bias[user_id] + self.item_bias[item_id]
        return res
    
    # Uses user_items CSR matrix to skip movies that user has already watched
    def recommend(self, user_id: int, user_items: sp.csr_matrix, n_top=10) -> List[Tuple[int, float]]:
        predictions = []
        watched = user_items[user_id].indices
        for item_id in range(self.n_items):
            if item_id in watched:
                continue
            score = self.predict_score(user_id, item_id)
            predictions.append((item_id, score))
        predictions.sort(key=lambda pr: -pr[1])
        return predictions[:n_top]

    def similar_items(self, item_id: int, n_top=10) -> List[Tuple[int, float]]:
        scores = np.dot(self.item_factors, self.item_factors[item_id])
        scores = scores / np.linalg.norm(self.item_factors[item_id])
        scores = scores / np.linalg.norm(self.item_factors, axis=-1)
        predictions = list(enumerate(scores))
        predictions.sort(key=lambda pr: -pr[1])
        return predictions[:n_top]


### Задание 1. Не используя готовые решения, реализовать SVD разложение, используя SGD на explicit данных

*Loss = min sum_{u,i} [(R_ui - (<v_i, w_u> + vb_u + wb_i + gb))^2 + l2_regularization]*

In [107]:
class SgdModule(RecModule):
    def __init__(self, n_factors=100, lr=0.1, user_mu=0.01, item_mu=0.01):
        super().__init__(n_factors, lr, user_mu, item_mu)
    
    def fit(self, ratings: pd.core.frame.DataFrame, n_epochs=15, info_every=2):
        self.n_users = ratings['user_id'].max() + 1
        self.n_items = ratings['movie_id'].max() + 1
        self._init_vectors()
        self.global_bias = np.mean(ratings['rating'])
        print(f'Mean rating is {self.global_bias}')
        n_data = len(ratings)

        for epoch in range(n_epochs):
            df = ratings.sample(frac=1)
            loss = 0.
            for user_id, item_id, rating in zip(ratings['user_id'], ratings['movie_id'], ratings['rating']):
                error = self.predict_score(user_id, item_id) - rating
                self.user_factors[user_id] -= self.lr * (error * self.item_factors[item_id] 
                                                + self.user_mu * self.user_factors[user_id])
                self.item_factors[item_id] -= self.lr * (error * self.user_factors[user_id]
                                                + self.item_mu * self.item_factors[item_id])
                self.user_bias[user_id] -= self.lr * (error + self.user_mu * self.user_bias[user_id])
                self.item_bias[item_id] -= self.lr * (error + self.item_mu * self.item_bias[item_id])
                if epoch % info_every == 0 or epoch + 1 == n_epochs:
                    loss += error ** 2
            if epoch % info_every == 0 or epoch + 1 == n_epochs:
                loss /= n_data
                print(f'Epoch {epoch} finished with loss {loss}')

In [108]:
sgd_model = SgdModule(N_FACTORS, LEARNING_RATE)
sgd_model.fit(ratings, N_EPOCHS)


Mean rating is 3.581564453029317
Epoch 0 finished with loss 0.9133513162898419
Epoch 2 finished with loss 0.7688043791726713
Epoch 4 finished with loss 0.6710600393622799
Epoch 6 finished with loss 0.5831216310751124
Epoch 8 finished with loss 0.517075968305926
Epoch 9 finished with loss 0.4913656284997129


In [114]:
get_similars(MOVIE_ID, sgd_model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '591    Beauty and the Beast (1991)',
 '360    Lion King, The (1994)',
 '2012    Little Mermaid, The (1989)',
 '1250    Back to the Future (1985)',
 '33    Babe (1995)',
 '1838    Mulan (1998)']

In [120]:
get_recommendations(USER_ID, sgd_model)

['1258    Young Frankenstein (1974)',
 '892    Rear Window (1954)',
 '1186    Lawrence of Arabia (1962)',
 '740    Dr. Strangelove or: How I Learned to Stop Worr...',
 '2961    Yojimbo (1961)',
 '1264    Big Sleep, The (1946)',
 '886    Philadelphia Story, The (1940)',
 '1211    Annie Hall (1977)',
 '901    Maltese Falcon, The (1941)',
 '2836    Sanjuro (1962)']


### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

*Loss = min sum_{u,i} [C_ui * (P_ui - <v_i, w_u>)^2 + l2_regularization]*

*C_ui = 1 + confidence * R_ui*

*P = implicit(R)*

In [246]:
from numpy.linalg import solve

class AlsModule(RecModule):
    def __init__(self, n_factors=100, lr=0.1, user_mu=0.01, item_mu=0.01, confidence = 20):
        super().__init__(n_factors, lr, user_mu, item_mu)
        self.c = confidence

    def fit_new(self, user_items: sp.csr_matrix, n_epochs=15, info_every=2):
        self.n_users, self.n_items = user_items.shape
        self._init_vectors()
        self.fit(user_items, n_epochs, info_every)
    
    def fit(self, user_items: sp.csr_matrix, n_epochs=15, info_every=2):
        confidence = self.c * user_items
        item_reg = self.n_users * self.item_mu
        user_reg = self.n_items * self.user_mu

        for epoch in range(n_epochs):
            VTV = np.dot(self.item_factors, self.item_factors.T).diagonal()
            for user_id in range(self.n_users):
                C_u = np.array(confidence[user_id].toarray()[0])
                A = np.dot(VTV, (C_u + 1)) + user_reg
                B = np.dot(self.item_factors.T, C_u + 1)
                self.user_factors[user_id] = B / A

            WTW = np.dot(self.user_factors, self.user_factors.T).diagonal()
            for item_id in range(self.n_items):
                C_u = np.array(confidence[:, item_id].toarray().T[0])
                A = np.dot(WTW, (C_u + 1)) + item_reg
                B = np.dot(self.user_factors.T, C_u + 1)
                self.item_factors[item_id] = B / A

            if epoch % info_every == 0 or epoch + 1 == n_epochs:
                loss = 0.
                for user_id, item_id in zip(*user_items.nonzero()):
                    error = 1. - self.predict_score(user_id, item_id)
                    loss += error ** 2
                n_data = user_items.count_nonzero()
                loss /= n_data
                print(f'Epoch {epoch} finished with loss {loss}')

In [247]:
als_model = AlsModule(n_factors=N_FACTORS)
als_model.fit_new(user_item_csr, n_epochs=N_EPOCHS)

Epoch 0 finished with loss 0.8496749585826605
Epoch 2 finished with loss 0.003013812931910703
Epoch 4 finished with loss 0.0016856554983405456
Epoch 6 finished with loss 0.0011383422738825786
Epoch 8 finished with loss 0.0008212011395344477
Epoch 9 finished with loss 0.0007104927439195254


In [301]:
get_similars(MOVIE_ID, als_model)

['0    Toy Story (1995)',
 '584    Aladdin (1992)',
 '38    Clueless (1995)',
 '3045    Toy Story 2 (1999)',
 '1250    Back to the Future (1985)',
 '33    Babe (1995)',
 '360    Lion King, The (1994)',
 "2286    Bug's Life, A (1998)",
 '1    Jumanji (1995)',
 "523    Schindler's List (1993)"]

In [256]:
get_recommendations(USER_ID, als_model) 

['2789    American Beauty (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '589    Silence of the Lambs, The (1991)',
 '2502    Matrix, The (1999)',
 '2693    Sixth Sense, The (1999)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '604    Fargo (1996)',
 '315    Shawshank Redemption, The (1994)',
 "523    Schindler's List (1993)",
 '585    Terminator 2: Judgment Day (1991)']

### Задание 3. Не используя готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных